In [45]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict
import json
from misc_functions import load_sig_chans, channel_names_to_indices


['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

In [46]:
# Initialize the outer dictionary.
subjects_electrodestoROIs_dict = {}

### make subjects rois to electrodes dict. Don't need to run this more than once.

In [ ]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071']

for sub in subjects:
    # sub = 'D0059'
    task = 'GlobalLocal'
    output_name = "Response_fixationCrossBase_1sec_mirror"
    events = ["Response"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False


    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    default_dict = gen_labels(good.info)
    
    # Create rawROI_dict for the subject
    rawROI_dict = defaultdict(list)
    for key, value in default_dict.items():
        rawROI_dict[value].append(key)
    rawROI_dict = dict(rawROI_dict)

    # Filter out keys containing "White-Matter"
    filtROI_dict = {key: value for key, value in rawROI_dict.items() if "White-Matter" not in key}

    # Store the dictionaries in the subjects dictionary
    subjects_electrodestoROIs_dict[sub] = {
        'default_dict': dict(default_dict),
        'rawROI_dict': dict(rawROI_dict),
        'filtROI_dict': dict(filtROI_dict)
    }


# Save to a JSON file
filename = 'subjects_electrodestoROIs_dict.json'
with open(filename, 'w') as file:
    json.dump(subjects_electrodestoROIs_dict, file, indent=4)

print(f"Saved subjects_dict to {filename}")

### load subjects electrodes to rois dict

In [3]:
# Load from a JSON file
filename = 'subjects_electrodestoROIs_dict.json'

with open(filename, 'r') as file:
    subjects_electrodestoROIs_dict = json.load(file)

print(f"Loaded data from {filename}")

Loaded data from subjects_electrodestoROIs_dict.json


### load high gamma data so we can do roi analysis on it
once we have more subjects, turn this into a function and loop over all subjects.  
this code is a crime against humanity

In [47]:
import os
import mne
import numpy as np


def load_mne_objects(sub, output_name, task, LAB_root=None):
    """
    Load MNE objects for a given subject and output name.

    Parameters:
    - sub (str): Subject identifier.
    - output_name (str): Output name used in the file naming.
    - task (str): Task identifier.
    - LAB_root (str, optional): Root directory for the lab. If None, it will be determined based on the OS.

    Returns:
    A dictionary containing loaded MNE objects.
    """

    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    
    # Ensure save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Define file paths
    HG_ev1_file = f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif'
    HG_base_file = f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif'
    HG_ev1_rescaled_file = f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif'

    # Load the objects
    HG_ev1 = mne.read_epochs(HG_ev1_file)
    HG_base = mne.read_epochs(HG_base_file)
    HG_ev1_rescaled = mne.read_epochs(HG_ev1_rescaled_file)
    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0))
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    return {
        'HG_ev1': HG_ev1,
        'HG_base': HG_base,
        'HG_ev1_rescaled': HG_ev1_rescaled,
        'HG_ev1_evoke': HG_ev1_evoke,
        'HG_ev1_evoke_rescaled': HG_ev1_evoke_rescaled
    }

# Example usage
# sub = 'D0057'
# output_name = "Stimulus_i25and75_fixationCrossBase_1sec_mirror"
# task = 'GlobalLocal'
loaded_objects_D0057_i = load_mne_objects('D0057', "Stimulus_i25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')
loaded_objects_D0057_c = load_mne_objects('D0057', "Stimulus_c25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')

loaded_objects_D0059_i = load_mne_objects('D0059', "Stimulus_i25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')
loaded_objects_D0059_c = load_mne_objects('D0059', "Stimulus_c25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')

loaded_objects_D0063_i = load_mne_objects('D0063', "Stimulus_i25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')
loaded_objects_D0063_c = load_mne_objects('D0063', "Stimulus_c25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')

loaded_objects_D0065_i = load_mne_objects('D0065', "Stimulus_i25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')
loaded_objects_D0065_c = load_mne_objects('D0065', "Stimulus_c25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')

loaded_objects_D0069_i = load_mne_objects('D0069', "Stimulus_i25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')
loaded_objects_D0069_c = load_mne_objects('D0069', "Stimulus_c25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')

loaded_objects_D0071_i = load_mne_objects('D0071', "Stimulus_i25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')
loaded_objects_D0071_c = load_mne_objects('D0071', "Stimulus_c25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')


# Access the objects
HG_ev1_D0057_i = loaded_objects_D0057_i['HG_ev1']
HG_base_D0057_i = loaded_objects_D0057_i['HG_base']
HG_ev1_rescaled_D0057_i = loaded_objects_D0057_i['HG_ev1_rescaled']
HG_ev1_evoke_D0057_i = loaded_objects_D0057_i['HG_ev1_evoke']
HG_ev1_evoke_rescaled_D0057_i = loaded_objects_D0057_i['HG_ev1_evoke_rescaled']

HG_ev1_D0057_c = loaded_objects_D0057_c['HG_ev1']
HG_base_D0057_c = loaded_objects_D0057_c['HG_base']
HG_ev1_rescaled_D0057_c = loaded_objects_D0057_c['HG_ev1_rescaled']
HG_ev1_evoke_D0057_c = loaded_objects_D0057_c['HG_ev1_evoke']
HG_ev1_evoke_rescaled_D0057_c = loaded_objects_D0057_c['HG_ev1_evoke_rescaled']

# HG_ev1_D0059_i = loaded_objects_D0059_i['HG_ev1']
# HG_base_D0059_i = loaded_objects_D0059_i['HG_base']
# HG_ev1_rescaled_D0059_i = loaded_objects_D0059_i['HG_ev1_rescaled']
# HG_ev1_evoke_D0059_i = loaded_objects_D0059_i['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0059_i = loaded_objects_D0059_i['HG_ev1_evoke_rescaled']

# HG_ev1_D0059_c = loaded_objects_D0059_c['HG_ev1']
# HG_base_D0059_c = loaded_objects_D0059_c['HG_base']
# HG_ev1_rescaled_D0059_c = loaded_objects_D0059_c['HG_ev1_rescaled']
# HG_ev1_evoke_D0059_c = loaded_objects_D0059_c['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0059_c = loaded_objects_D0059_c['HG_ev1_evoke_rescaled']

# HG_ev1_D0063_i = loaded_objects_D0063_i['HG_ev1']
# HG_base_D0063_i = loaded_objects_D0063_i['HG_base']
# HG_ev1_rescaled_D0063_i = loaded_objects_D0063_i['HG_ev1_rescaled']
# HG_ev1_evoke_D0063_i = loaded_objects_D0063_i['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0063_i = loaded_objects_D0063_i['HG_ev1_evoke_rescaled']

# HG_ev1_D0063_c = loaded_objects_D0063_c['HG_ev1']
# HG_base_D0063_c = loaded_objects_D0063_c['HG_base']
# HG_ev1_rescaled_D0063_c = loaded_objects_D0063_c['HG_ev1_rescaled']
# HG_ev1_evoke_D0063_c = loaded_objects_D0063_c['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0063_c = loaded_objects_D0063_c['HG_ev1_evoke_rescaled']

# HG_ev1_D0065_i = loaded_objects_D0065_i['HG_ev1']
# HG_base_D0065_i = loaded_objects_D0065_i['HG_base']
# HG_ev1_rescaled_D0065_i = loaded_objects_D0065_i['HG_ev1_rescaled']
# HG_ev1_evoke_D0065_i = loaded_objects_D0065_i['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0065_i = loaded_objects_D0065_i['HG_ev1_evoke_rescaled']

# HG_ev1_D0065_c = loaded_objects_D0065_c['HG_ev1']
# HG_base_D0065_c = loaded_objects_D0065_c['HG_base']
# HG_ev1_rescaled_D0065_c = loaded_objects_D0065_c['HG_ev1_rescaled']
# HG_ev1_evoke_D0065_c = loaded_objects_D0065_c['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0065_c = loaded_objects_D0065_c['HG_ev1_evoke_rescaled']

# HG_ev1_D0069_i = loaded_objects_D0069_i['HG_ev1']
# HG_base_D0069_i = loaded_objects_D0069_i['HG_base']
# HG_ev1_rescaled_D0069_i = loaded_objects_D0069_i['HG_ev1_rescaled']
# HG_ev1_evoke_D0069_i = loaded_objects_D0069_i['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0069_i = loaded_objects_D0069_i['HG_ev1_evoke_rescaled']

# HG_ev1_D0069_c = loaded_objects_D0069_c['HG_ev1']
# HG_base_D0069_c = loaded_objects_D0069_c['HG_base']
# HG_ev1_rescaled_D0069_c = loaded_objects_D0069_c['HG_ev1_rescaled']
# HG_ev1_evoke_D0069_c = loaded_objects_D0069_c['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0069_c = loaded_objects_D0069_c['HG_ev1_evoke_rescaled']

# HG_ev1_D0071_i = loaded_objects_D0071_i['HG_ev1']
# HG_base_D0071_i = loaded_objects_D0071_i['HG_base']
# HG_ev1_rescaled_D0071_i = loaded_objects_D0071_i['HG_ev1_rescaled']
# HG_ev1_evoke_D0071_i = loaded_objects_D0071_i['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0071_i = loaded_objects_D0071_i['HG_ev1_evoke_rescaled']

# HG_ev1_D0071_c = loaded_objects_D0071_c['HG_ev1']
# HG_base_D0071_c = loaded_objects_D0071_c['HG_base']
# HG_ev1_rescaled_D0071_c = loaded_objects_D0071_c['HG_ev1_rescaled']
# HG_ev1_evoke_D0071_c = loaded_objects_D0071_c['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0071_c = loaded_objects_D0071_c['HG_ev1_evoke_rescaled']

Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_i25and75_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
224 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_i25and75_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_i25and75_fixationCrossBase_1sec_mirror_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
     

### load evoked and stuff for all subjects in a dictionary

In [8]:
import os
import mne
import numpy as np

def create_subjects_mne_objects_dict(subjects, output_names, task, LAB_root=None):
    subjects_mne_objects = {}

    for sub in subjects:
        print(f"Loading data for subject: {sub}")  # Debugging print
        sub_mne_objects = {}
        for output_name in output_names:
            print(f"  Loading output: {output_name}")  # Debugging print
            mne_objects = load_mne_objects(sub, output_name, task, LAB_root)

            # Debugging prints for data shapes
            print(f"    HG_ev1 shape: {mne_objects['HG_ev1'].get_data().shape}")
            print(f"    HG_base shape: {mne_objects['HG_base'].get_data().shape}")
            print(f"    HG_ev1_rescaled shape: {mne_objects['HG_ev1_rescaled'].get_data().shape}")
            print(f"    HG_ev1_evoke shape: {mne_objects['HG_ev1_evoke'].data.shape}")
            print(f"    HG_ev1_evoke_rescaled shape: {mne_objects['HG_ev1_evoke_rescaled'].data.shape}")

            sub_mne_objects[output_name] = mne_objects
        subjects_mne_objects[sub] = sub_mne_objects

    return subjects_mne_objects

# Example usage
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071']
output_names = ["Stimulus_i25and75_fixationCrossBase_1sec_mirror", "Stimulus_c25and75_fixationCrossBase_1sec_mirror"]
task = 'GlobalLocal'

subjects_mne_objects = create_subjects_mne_objects_dict(subjects, output_names, task)

Loading data for subject: D0057
  Loading output: Stimulus_i25and75_fixationCrossBase_1sec_mirror
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_i25and75_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
224 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_i25and75_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_i25and

### load stimulus significant channels. Compare ROI electrodes in next cell to these to see if they're included.

maybe do response significant channels too/instead?

In [9]:
def get_sig_chans(sub, task, LAB_root=None):
    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

    stim_filename = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror.json'
    stim_sig_chans = load_sig_chans(stim_filename)
    return stim_sig_chans


# List of subjects
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071']

# Initialize an empty dictionary to store significant channels per subject
sig_chans_per_subject = {}

# Populate the dictionary using get_sig_chans for each subject
for sub in subjects:
    sig_chans_per_subject[sub] = get_sig_chans(sub, 'GlobalLocal')

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071


dlPFC based on Yamagishi et al 2016 definition is G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup

In [10]:
dlpfc_electrodes_per_subject = {}

for sub in subjects_electrodestoROIs_dict:
    dlpfc = {key: value for key, value in subjects_electrodestoROIs_dict[sub]['filtROI_dict'].items() 
             if any(roi in key for roi in ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"])}

    # Aggregate electrodes into a list for each subject
    dlpfc_electrodes = []
    for electrodes in dlpfc.values():
        dlpfc_electrodes.extend(electrodes)

    dlpfc_electrodes_per_subject[sub] = dlpfc_electrodes
    print(f'For subject {sub}, dlpfc electrodes are: {dlpfc_electrodes}')

# dlpfc_electrodes_per_subject now contains the list of electrodes for each subject

# Assuming sig_chans_per_subject is a dictionary with subjects as keys and lists of significant channels as values

sig_dlpfc_electrodes_per_subject = {}

for sub, dlpfc_electrodes in dlpfc_electrodes_per_subject.items():
    # Retrieve the list of significant channels for the subject
    sig_chans = sig_chans_per_subject.get(sub, [])

    # Find the intersection of DLPFC electrodes and significant channels for the subject
    sig_dlpfc_electrodes = [elec for elec in dlpfc_electrodes if elec in sig_chans]

    # Store the significant DLPFC electrodes for the subject
    sig_dlpfc_electrodes_per_subject[sub] = sig_dlpfc_electrodes

    # Print results for each subject
    print(f"Subject {sub} significant DLPFC electrodes: {sig_dlpfc_electrodes}")

For subject D0057, dlpfc electrodes are: ['RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12']
For subject D0059, dlpfc electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, dlpfc electrodes are: ['LASF10', 'LASF14', 'LASF15', 'LASF16', 'LMSF5', 'LMSF6', 'LMSF12', 'LPSF10', 'LPSF12', 'RAI10', 'RAI11', 'RAI16', 'RAMF11', 'RAMF12', 'RAMF13', 'RMMF13', 'RMMF14', 'RASF15', 'RMSF8', 'RMSF9', 'RMSF10', 'RMSF7', 'RAMF8', 'RAMF9', 'RAMF10', 'RMMF9', 'RMMF10']
For subject D0065, dlpfc electrodes are: ['RASF13', 'RASF14', 'RASF15', 'RMSF11', 'RMSF12', 'RMSF13', 'RMSF14']
For subject D0069, dlpfc electrodes are: []
For subject D0071, dlpfc electrodes are: ['RIA11', 'RIA12', 'RIA14', 'RIA16', 'RIP14', 'RIP15']
Subject D0057 significant DLPFC electrodes: ['RPI14']
Subject D0059 significant DLPFC electrodes: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
Subject D0063 significant DLPFC electrodes: ['LMSF5', 'LPSF12', 'RAMF12', 'RMMF13', 'RMMF14', 'RMMF10']
Subject

### make plot of only specific channels, averaged together

this is across subjects which is wrong. Delete later.

In [12]:
# import numpy as np

# def compute_average_and_sem(evoked_rescaled_data, selected_channels):
#     """
#     Compute the average and standard error of the mean for selected channels in an Evoked object.

#     Parameters:
#     - evoked_rescaled_data (mne.Evoked): The Evoked object containing the data.
#     - selected_channels (list of str): A list of channel names to include in the computation.

#     Returns:
#     - average_data (numpy.ndarray): The average data across the selected channels.
#     - sem_data (numpy.ndarray): The standard error of the mean for the selected channels.
#     """

#     # Find the indices of these channels in the Evoked object
#     channel_indices = [evoked_rescaled_data.ch_names.index(ch) for ch in selected_channels if ch in evoked_rescaled_data.ch_names]

#     # Check if all channels were found
#     if len(channel_indices) != len(selected_channels):
#         missing_ch = set(selected_channels) - set(evoked_rescaled_data.ch_names)
#         raise ValueError(f"Some channels not found in Evoked object: {missing_ch}")

#     # Extract data for these channels
#     selected_data = evoked_rescaled_data.data[channel_indices, :]

#     # Compute the average and standard error
#     average_data = selected_data.mean(axis=0)
#     sem_data = selected_data.std(axis=0) / np.sqrt(len(channel_indices))

#     return average_data, sem_data

# # Example usage
# average_mPFC_D0057_i, sem_mPFC_D0057_i = compute_average_and_sem(HG_ev1_evoke_rescaled_D0057_i, ['RPI14', 'RAMF10', 'RAMF11', 'RAMF12'])
# average_mPFC_D0057_c, sem_mPFC_D0057_c = compute_average_and_sem(HG_ev1_evoke_rescaled_D0057_c, ['RPI14', 'RAMF10', 'RAMF11', 'RAMF12'])
# def compute_average_and_sem_subject(subjects_mne_objects, sub, output_name, sig_roi_electrodes_per_subject):
#     """
#     Compute the average and standard error of the mean for a subject's significant roi electrodes.

#     Parameters:
#     - subjects_mne_objects (dict): Dictionary containing MNE objects for each subject.
#     - sub (str): Subject ID.
#     - output_name (str): Output name corresponding to the specific analysis.
#     - sig_roi_electrodes_per_subject (dict): Dictionary containing significant roi electrodes for each subject.

#     Returns:
#     - average_data (numpy.ndarray): The average data across the significant roi electrodes for the subject.
#     - sem_data (numpy.ndarray): The standard error of the mean for the significant roi electrodes for the subject.
#     """
#     selected_channels = sig_roi_electrodes_per_subject.get(sub, [])
#     sub_objects = subjects_mne_objects.get(sub, {})
#     evoked_rescaled_data = sub_objects.get(output_name, {}).get('HG_ev1_evoke_rescaled')

#     if not evoked_rescaled_data or not selected_channels:
#         return None, None

#     return compute_average_and_sem(evoked_rescaled_data, selected_channels)

# def compute_overall_average_and_sem(average_sem_data):
#     """
#     Compute the overall average and standard error of the mean across subjects.

#     Parameters:
#     - average_sem_data (list of tuples): List of tuples containing average and SEM data for each subject.

#     Returns:
#     - overall_average (numpy.ndarray): The overall average data across subjects.
#     - overall_sem (numpy.ndarray): The overall standard error of the mean across subjects.
#     """
#     all_averages = np.array([avg for avg, sem in average_sem_data if avg is not None])
#     overall_average = all_averages.mean(axis=0)

#     # Compute overall SEM using pooled standard deviation
#     all_sems = np.array([sem for avg, sem in average_sem_data if sem is not None])
#     overall_sem = np.sqrt(np.sum(all_sems**2, axis=0)) / len(all_sems)

#     return overall_average, overall_sem

# # Compute averages and SEMs for each subject and condition
# average_sem_dlPFC_data_congruent = []
# average_sem_dlPFC_data_incongruent = []

# for sub in dlpfc_electrodes_per_subject.keys():
#     for output_name in output_names:
#         avg, sem = compute_average_and_sem_subject(subjects_mne_objects, sub, output_name, dlpfc_electrodes_per_subject)
#         if avg is not None and sem is not None:
#             if "_c" in output_name:  # Replace with actual identifier
#                 average_sem_dlPFC_data_congruent.append((avg, sem))
#             elif "_i" in output_name:  # Replace with actual identifier
#                 average_sem_dlPFC_data_incongruent.append((avg, sem))

# # Compute the separate averages and SEMs for congruent and incongruent conditions
# overall_average_dlPFC_congruent, overall_sem_dlPFC_congruent = compute_overall_average_and_sem(average_sem_dlPFC_data_congruent)
# overall_average_dlPFC_incongruent, overall_sem_dlPFC_incongruent = compute_overall_average_and_sem(average_sem_dlPFC_data_incongruent)


### do stats

current approach is to run time_perm_cluster on significant dlpfc electrodes for each subject, comparing congruent and incongruent conditions. Then, average p-values across all subjects. Discuss this with Greg, probably wrong approach.

**1/23 new approach is to average across all trials for sig dlpfc electrodes, comparing incongruent and congruent conditions. Then, run stats on this new avg electrode value x time array.

Also, I'm using HG_ev1_rescaled instead of HG_ev1 to compare congruent and incongruent, so that they're normalized with a common baseline. I think this is better than comparing the raw HG traces directly.

### this is avg across electrode, don't do that. Avg across timepoints instead. Delete this once the other method works.

### this is 1/23 new approach of avg across trials first

do stats and plotting together. Stats needs trial avg data, plotting just needs congruent_data without trial averaging (initially at least)

In [16]:
congruent_epochs = subjects_mne_objects['D0063']['Stimulus_c25and75_fixationCrossBase_1sec_mirror']['HG_ev1_rescaled'].pick_channels(dlpfc_electrodes_per_subject['D0063'])
print(congruent_epochs.get_data().shape)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
(224, 27, 5121)


C:\Users\jz421\AppData\Local\Temp\ipykernel_10912\974801574.py:1: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  congruent_epochs = subjects_mne_objects['D0063']['Stimulus_c25and75_fixationCrossBase_1sec_mirror']['HG_ev1_rescaled'].pick_channels(dlpfc_electrodes_per_subject['D0063'])


In [75]:
import numpy as np

# Initialize lists to store data
congruent_data_trialAvg_list = []
incongruent_data_trialAvg_list = []

for sub in subjects:
    # Skip this subject if no dlpfc electrodes
    if not sig_dlpfc_electrodes_per_subject[sub]:
        continue

    # Load trial-level data for each condition and pick significant DLPFC electrodes
    congruent_epochs = subjects_mne_objects[sub]['Stimulus_c25and75_fixationCrossBase_1sec_mirror']['HG_ev1_rescaled'].pick_channels(sig_dlpfc_electrodes_per_subject[sub])
    incongruent_epochs = subjects_mne_objects[sub]['Stimulus_i25and75_fixationCrossBase_1sec_mirror']['HG_ev1_rescaled'].pick_channels(sig_dlpfc_electrodes_per_subject[sub])

    # Average across the trials dimension, ignoring NaNs
    congruent_data_trialAvg_list.append(np.nanmean(congruent_epochs.get_data(), axis=0))
    incongruent_data_trialAvg_list.append(np.nanmean(incongruent_epochs.get_data(), axis=0))

# Concatenate data across all subjects
congruent_data_trialAvg = np.concatenate(congruent_data_trialAvg_list, axis=0)
incongruent_data_trialAvg = np.concatenate(incongruent_data_trialAvg_list, axis=0)

# Calculate mean and SEM across electrodes
overall_average_dlPFC_congruent = np.nanmean(congruent_data_trialAvg, axis=0)
overall_sem_dlPFC_congruent = np.std(congruent_data_trialAvg, axis=0, ddof=1) / np.sqrt(congruent_data_trialAvg.shape[0])
overall_average_dlPFC_incongruent = np.nanmean(incongruent_data_trialAvg, axis=0)
overall_sem_dlPFC_incongruent = np.std(incongruent_data_trialAvg, axis=0, ddof=1) / np.sqrt(incongruent_data_trialAvg.shape[0])

# Run the permutation test
mat = time_perm_cluster(congruent_data_trialAvg, incongruent_data_trialAvg, 0.05, n_jobs=6)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\jz421\AppData\Local\Temp\ipykernel_10912\3644521502.py:13: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  congruent_epochs = subjects_mne_objects[sub]['Stimulus_c25and75_fixationCrossBase_1sec_mirror']['HG_ev1_rescaled'].pick_channels(sig_dlpfc_electrodes_per_subject[sub])
C:\Users\jz421\AppData\Local\Temp\ipykernel_10912\3644521502.py:14: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  incongruent_epochs = subjects_mne_objects[sub]['Stimulus_i25and75_fixationCrossBase_1sec_mirror']['HG_ev1_rescaled'].pick_channels(sig_dlpfc_elect

### plot and QC stats

check if no significant differences..

In [27]:
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(mat)
plt.xlabel('Timepoints')
plt.ylabel('Significance (0 or 1)')
plt.title('Permutation Test Significance Over Time')
plt.show()

KeyboardInterrupt: 

try a basic t-test as a sanity check (ok its also not significant)

In [37]:
import numpy as np
from scipy.stats import ttest_ind

p_values = np.array([ttest_ind(congruent_data[:, tp], incongruent_data[:, tp]).pvalue for tp in range(congruent_data.shape[1])])
significance = p_values < 0.05  # Apply a significance threshold, e.g., p < 0.05

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(range(congruent_data.shape[1]), significance, label='Significance (p < 0.05)')
plt.xlabel('Time Point')
plt.ylabel('Significance (True/False)')
plt.title('Statistical Significance Over Time')
plt.legend()
plt.show()


KeyboardInterrupt: 

now aggregate the significance test results for each subject across subjects. Talk to greg about how to do this...vote count is probably wrong

In [ ]:
# Assuming subject_results is a dictionary with keys as subject IDs and values as binary arrays indicating significant time points
num_subjects = len(subject_results)
num_time_points = len(subject_results[next(iter(subject_results))])  # Assuming all arrays are the same length

# Initialize an array to count votes
vote_counts = np.zeros(num_time_points)

# Count votes
for sub in subject_results:
    vote_counts += subject_results[sub]

# Determine significance based on a threshold (e.g., at least 50% of subjects show significance)
threshold = 0.5 * num_subjects  # 50% of subjects
significant_time_points = vote_counts >= threshold


### plot example with mpfc, can delete

In [ ]:
import matplotlib.pyplot as plt

# Use the times from your evoked data
times = HG_ev1_evoke_rescaled_D0057_i.times  # Modify as needed to match your data

plt.figure(figsize=(10, 6))

# Plot average_mPFC_D0057_i with SEM shading
plt.plot(times, average_mPFC_D0057_i, label='Average mPFC Incongruent')
plt.fill_between(times, average_mPFC_D0057_i - sem_mPFC_D0057_i, average_mPFC_D0057_i + sem_mPFC_D0057_i, alpha=0.3)

# Plot average_mPFC_D0057_c with SEM shading
plt.plot(times, average_mPFC_D0057_c, label='Average mPFC Congruent')
plt.fill_between(times, average_mPFC_D0057_c - sem_mPFC_D0057_c, average_mPFC_D0057_c + sem_mPFC_D0057_c, alpha=0.3)

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average mPFC Signal with Standard Error for D0057')
plt.legend()
plt.savefig(save_dir + f'_mPFC_congruency_zscore_D0057.png')
plt.show()


### plot dlpfc inc vs. con, avg across all subjects

In [76]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_dlPFC_congruency_zscore.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0063_c = subjects_mne_objects['D0063']['Stimulus_c25and75_fixationCrossBase_1sec_mirror']['HG_ev1_evoke_rescaled']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0063_c.times  # Modify as needed to match your data

# Plot overall_average_dlPFC_congruent with SEM shading
plt.plot(times, overall_average_dlPFC_congruent, label='Average dlPFC Congruent')
plt.fill_between(times, overall_average_dlPFC_congruent - overall_sem_dlPFC_congruent, 
                 overall_average_dlPFC_congruent + overall_sem_dlPFC_congruent, alpha=0.3)

# Plot overall_average_dlPFC_incongruent with SEM shading
plt.plot(times, overall_average_dlPFC_incongruent, label='Average dlPFC Incongruent')
plt.fill_between(times, overall_average_dlPFC_incongruent - overall_sem_dlPFC_incongruent, 
                 overall_average_dlPFC_incongruent + overall_sem_dlPFC_incongruent, alpha=0.3)

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average dlPFC Signal with Standard Error (Congruent vs Incongruent)')
plt.legend()
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


KeyboardInterrupt: 